# Edit Line Parameters via CSV/YAML

This notebook demonstrates a more human-friendly workflow for viewing and editing the line parameter list using CSV or YAML, then converting back to the FITS table consumed by qsofitmore.

- Exports existing `qsopar.fits` to `qsopar.csv` and optionally `qsopar.yaml`.
- You can edit the CSV/YAML in any editor or spreadsheet.
- Converts the edited CSV/YAML back to FITS with the exact schema expected by the fitter.

Columns preserved (identical to the original schema):
`lambda, compname, minwav, maxwav, linename, ngauss, inisig, minsig, maxsig, voff, vindex, windex, findex, fvalue`.

In [1]:
from astropy.table import Table
from astropy.io import fits
import os, sys, pathlib

try:
    from qsofitmore.line_params_io import (
        fits_to_csv, csv_to_fits,
        fits_to_yaml, yaml_to_fits,
        write_template_csv, write_template_yaml
    )
except ModuleNotFoundError:
    # Add repo root to sys.path (works when running from examples/)
    here = pathlib.Path.cwd()
    candidates = [here, here.parent, here.parent.parent, here.parent.parent.parent]
    for pth in candidates:
        if (pth / 'qsofitmore' / 'line_params_io.py').exists():
            sys.path.insert(0, str(pth))
            break
    from qsofitmore.line_params_io import (
        fits_to_csv, csv_to_fits,
        fits_to_yaml, yaml_to_fits,
        write_template_csv, write_template_yaml
    )

path = './output/'  # keep consistent with other examples
fits_path = os.path.join(path, 'qsopar.fits')
csv_path = os.path.join(path, 'qsopar.csv')
yaml_path = os.path.join(path, 'qsopar.yaml')

print('FITS:', fits_path)
print('CSV :', csv_path)
print('YAML:', yaml_path)

FITS: ./output/qsopar.fits
CSV : ./output/qsopar.csv
YAML: ./output/qsopar.yaml


## Export current FITS to CSV/YAML

If `qsopar.fits` exists, export it to CSV (always) and YAML (if `pyyaml` is installed).
YAML is optional but nice for readability and comments.

In [2]:
if os.path.exists(fits_path):
    print('Exporting FITS -> CSV ...')
    fits_to_csv(fits_path, csv_path)
    print('Wrote', csv_path)

    # Try YAML export if PyYAML is available
    try:
        fits_to_yaml(fits_path, yaml_path)
        print('Wrote', yaml_path)
    except Exception as e:
        print('YAML export skipped:', e)
else:
    print('No FITS file found at', fits_path)
    print('You can still create blank templates below.')

Exporting FITS -> CSV ...
Wrote ./output/qsopar.csv
Wrote ./output/qsopar.yaml


## Preview CSV

Load the CSV into an Astropy Table to inspect the column order and a few rows.

In [3]:
if os.path.exists(csv_path):
    t = Table.read(csv_path, format='csv')
    print(t.colnames)
    t[:5]  # display first 5 rows
else:
    print('CSV not found. Run the export cell above or create a template below.')

['lambda', 'compname', 'minwav', 'maxwav', 'linename', 'ngauss', 'inisig', 'minsig', 'maxsig', 'voff', 'vindex', 'windex', 'findex', 'fvalue']


## Create CSV/YAML templates (optional)

If you want to start from scratch or add a new set, use these helpers to write empty templates you can fill in by hand.

In [4]:
# Uncomment to generate blank templates
# write_template_csv(csv_path)
# try:
#     write_template_yaml(yaml_path)
# except Exception as e:
#     print('YAML template skipped:', e)
# print('Templates written to', path)

## Edit the CSV/YAML

- Open `qsopar.csv` (or `qsopar.yaml`) in your editor or spreadsheet.
- Keep the header columns unchanged.
- Save your edits back to the same file.

Field notes:
- `lambda`: vacuum wavelength in Angstroms.
- `compname`: short component group name (e.g. Ha, Hb, MgII).
- `minwav`/`maxwav`: fitting window for the complex.
- `linename`: unique line identifier (e.g. OIII5007).
- `ngauss`: number of Gaussians for the line.
- `inisig`/`minsig`/`maxsig`: sigma ranges (in ln(lambda)).
- `voff`: velocity offset limit (ln(lambda)).
- `vindex`/`windex`: tie indices (>0 ties among same index).
- `findex`/`fvalue`: flux ratio ties (same index shares fixed ratios).

## Convert edited CSV/YAML back to FITS

This produces `qsopar.fits` with the exact schema expected by the fitter. If an original FITS exists, its header comments are preserved.

In [5]:
orig_header = None
if os.path.exists(fits_path):
    with fits.open(fits_path) as hdul:
        # Assume the parameter table is the first BinTable extension
        for hdu in hdul:
            if isinstance(hdu, fits.BinTableHDU):
                orig_header = hdu.header
                break

if os.path.exists(csv_path):
    print('CSV -> FITS ...')
    csv_to_fits(csv_path, fits_path, header=orig_header)
    print('Wrote', fits_path)
elif os.path.exists(yaml_path):
    print('YAML -> FITS ...')
    try:
        yaml_to_fits(yaml_path, fits_path, header=orig_header)
        print('Wrote', fits_path)
    except Exception as e:
        print('YAML import failed:', e)
else:
    print('Neither CSV nor YAML found to import.')

CSV -> FITS ...
Wrote ./output/qsopar.fits


## Sanity check

Open the new FITS and show a quick preview.

In [6]:
if os.path.exists(fits_path):
    t2 = Table.read(fits_path)
    print('Rows:', len(t2))
    t2[:5]
else:
    print('FITS not found. Did the conversion step succeed?')

Rows: 20
